# Imports

In [1]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using cuDNN version 5103 on context None
ERROR (theano.gpuarray): Could not initialize pygpu, support disabled
Traceback (most recent call last):
  File "/home/dbashir/anaconda2/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 227, in <module>
    use(config.device)
  File "/home/dbashir/anaconda2/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 214, in use
    init_dev(device, preallocate=preallocate)
  File "/home/dbashir/anaconda2/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 140, in init_dev
    context.free_gmem / MB))
RuntimeError: Trying to preallocate 10867 MB of GPU memory while only 342 MB are available.
Using Theano backend.


# Setup

In [2]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read()
print('corpus length:', len(text))

corpus length: 600901


In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('corpus length:', len(text))

corpus length: 600901


In [4]:
chars.insert(0, "\0")

In [5]:
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz'

In [6]:
char_indices = dict((c,i) for i,c in enumerate(chars))
indices_char = dict((i,c) for i,c in enumerate(chars))

In [7]:
idx = [char_indices[c] for c in text]

In [8]:
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [9]:
''.join(indices_char[i] for i in idx[:70])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not gro'

# 3 char model

## Create Inputs

In [10]:
cs=3
c1_dat = [idx[i] for i in xrange(0,len(idx)-1-cs, cs)]
c2_dat = [idx[i+1] for i in xrange(0,len(idx)-1-cs, cs)]
c3_dat = [idx[i+2] for i in xrange(0,len(idx)-1-cs, cs)]
c4_dat = [idx[i+3] for i in xrange(0,len(idx)-1-cs, cs)]

We create inputs by stacking info from ck_dat into an array

In [11]:
x1 = np.stack(c1_dat[:-2])
x2 = np.stack(c2_dat[:-2])
x3 = np.stack(c3_dat[:-2])

Output

In [12]:
y = np.stack(c4_dat[:-2])

In [13]:
x1[:4], x2[:4], x3[:4]

(array([40, 30, 29,  1]), array([42, 25,  1, 43]), array([29, 27,  1, 45]))

In [14]:
y[:4]

array([30, 29,  1, 40])

In [15]:
x1.shape, y.shape

((200297,), (200297,))

In [16]:
n_fac = 42 # num latent factors

In [17]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name)
    emb = Embedding(n_in, n_out, input_length=1)(inp)
    return inp, Flatten()(emb)

In [18]:
c1_in, c1 = embedding_input('c1', vocab_size, n_fac)
c2_in, c2 = embedding_input('c2', vocab_size, n_fac)
c3_in, c3 = embedding_input('c3', vocab_size, n_fac)

## Create, Train Model

In [19]:
n_hidden = 256 #size for hidden state

In [20]:
#input to hidden
dense_in = Dense(n_hidden, activation='relu')

In [21]:
#first hidden act is this applied to c1
c1_hidden = dense_in(c1)

In [22]:
dense_hidden = Dense(n_hidden, activation='tanh')

Make c2's dense, hidden layer 2 from c1 and add them

In [23]:
c2_dense = dense_in(c2)
hidden_2 = dense_hidden(c1_hidden)
c2_hidden = merge([c2_dense, hidden_2])

In [24]:
c3_dense = dense_in(c3)
hidden_3 = dense_hidden(c2_hidden)
c3_hidden = merge([c3_dense, hidden_3])

Now let's make the output layer

In [25]:
dense_out = Dense(vocab_size, activation='softmax')

That 3rd hidden state is the input to our output layer

In [26]:
c4_out = dense_out(c3_hidden)

In [27]:
model = Model([c1_in,c2_in,c3_in], c4_out)

In [28]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [29]:
model.optimizer.lr = 0.000001

In [30]:
model.fit([x1,x2,x3], y, batch_size=64, nb_epoch=4) #x1-x3 and y are the data we scraped

Epoch 1/4
200297/200297 [==============================] - 11s - loss: 4.4138    
Epoch 2/4
200297/200297 [==============================] - 11s - loss: 4.2851    
Epoch 3/4
200297/200297 [==============================] - 10s - loss: 4.0109    
Epoch 4/4
200297/200297 [==============================] - 10s - loss: 3.6198    


In [31]:
model.optimizer.lr = 0.01

In [32]:
model.fit([x1,x2,x3], y, batch_size=64, nb_epoch=4)

Epoch 1/4
200297/200297 [==============================] - 10s - loss: 3.3393    
Epoch 2/4
200297/200297 [==============================] - 11s - loss: 3.2115    
Epoch 3/4
200297/200297 [==============================] - 10s - loss: 3.1523    
Epoch 4/4
200297/200297 [==============================] - 10s - loss: 3.1241    


In [33]:
model.optimizer.lr = 0.000001

In [34]:
model.fit([x1,x2,x3], y, batch_size=64, nb_epoch=4)

Epoch 1/4
200297/200297 [==============================] - 11s - loss: 3.1081    
Epoch 2/4
200297/200297 [==============================] - 10s - loss: 3.0970    
Epoch 3/4
200297/200297 [==============================] - 11s - loss: 3.0882    
Epoch 4/4
200297/200297 [==============================] - 11s - loss: 3.0806    


In [35]:
model.optimizer.lr = 0.01

In [36]:
model.fit([x1,x2,x3], y, batch_size=64, nb_epoch=4)

Epoch 1/4
200297/200297 [==============================] - 10s - loss: 3.0736    
Epoch 2/4
200297/200297 [==============================] - 11s - loss: 3.0670    
Epoch 3/4
200297/200297 [==============================] - 11s - loss: 3.0604    
Epoch 4/4
200297/200297 [==============================] - 10s - loss: 3.0537    


## Test Model

In [37]:
def get_next(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs] #newaxis expands the tuple by 1 dimension
    p = model.predict(arrs)
    i = np.argmax(p) #get the biggest prediction
    return chars[i]

In [38]:
get_next('phi')

' '

## First RNN

In [39]:
cs=8 #size of unrolled RNN

In [40]:
#create a list of every 8th char, this is input
c_in_dat = [[idx[i+n] for i in xrange(0,len(idx)-1-cs, cs)]
            for n in range(cs)]

In [41]:
c_out_dat = [idx[i+cs] for i in xrange(0,len(idx)-1-cs, cs)]

In [42]:
xs = [np.stack(c[:-2]) for c in c_in_dat]

In [43]:
y = np.stack(c_out_dat[:-2])

In [44]:
[xs[n][:cs] for n in range(cs)]

[array([40,  1, 33,  2, 72, 67, 73,  2]),
 array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67])]

In [45]:
n_fac = 42

### Create and Train Model

In [46]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name)
    emb = Embedding(n_in, n_out, input_length=1)(inp)
    return inp, Flatten()(emb)

In [47]:
c_ins = [embedding_input('c'+str(n), vocab_size, n_fac) for n in range(cs)]

In [48]:
n_hidden = 256

In [49]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax')

In [50]:
hidden = dense_in(c_ins[0][1])

In [51]:
for i in range(1,cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden])

In [52]:
c_out = dense_out(hidden)

In [53]:
model = Model([c[0] for c in c_ins], c_out)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [54]:
model.fit(xs, y, batch_size=64, nb_epoch=12)

Epoch 1/12
75110/75110 [==============================] - 6s - loss: 2.5360     
Epoch 2/12
75110/75110 [==============================] - 6s - loss: 2.2559     
Epoch 3/12
75110/75110 [==============================] - 7s - loss: 2.1499     
Epoch 4/12
75110/75110 [==============================] - 9s - loss: 2.0792     
Epoch 5/12
75110/75110 [==============================] - 8s - loss: 2.0229     
Epoch 6/12
75110/75110 [==============================] - 8s - loss: 1.9772     
Epoch 7/12
75110/75110 [==============================] - 8s - loss: 1.9365     
Epoch 8/12
75110/75110 [==============================] - 9s - loss: 1.8986     
Epoch 9/12
75110/75110 [==============================] - 9s - loss: 1.8648     
Epoch 10/12
75110/75110 [==============================] - 8s - loss: 1.8364     
Epoch 11/12
75110/75110 [==============================] - 8s - loss: 1.8095     
Epoch 12/12
75110/75110 [==============================] - 8s - loss: 1.7854     


### Test Model

In [55]:
def get_next(inp):
    idxs = [np.array(char_indices[c])[np.newaxis] for c in inp]
    p = model.predict(idxs)
    return chars[np.argmax(p)]

In [56]:
get_next('for thos')

' '

## Keras RNN

In [57]:
n_hidden, n_fac, cs, vocab_size = (256, 42, 8, 86)

In [58]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs),
        SimpleRNN(n_hidden, activation='relu', inner_init='identity'),
        Dense(vocab_size, activation='softmax')
    ])

In [59]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_12 (Embedding)         (None, 8, 42)         3612        embedding_input_1[0][0]          
____________________________________________________________________________________________________
simplernn_1 (SimpleRNN)          (None, 256)           76544       embedding_12[0][0]               
____________________________________________________________________________________________________
dense_7 (Dense)                  (None, 86)            22102       simplernn_1[0][0]                
Total params: 102,258
Trainable params: 102,258
Non-trainable params: 0
____________________________________________________________________________________________________


In [60]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [61]:
model.fit(np.concatenate(xs,axis=1), y, batch_size=64, nb_epoch=8)

Epoch 1/8
75110/75110 [==============================] - 7s - loss: 2.8237     
Epoch 2/8
75110/75110 [==============================] - 7s - loss: 2.2990     
Epoch 3/8
75110/75110 [==============================] - 7s - loss: 2.0865     
Epoch 4/8
75110/75110 [==============================] - 7s - loss: 1.9470     
Epoch 5/8
75110/75110 [==============================] - 7s - loss: 1.8414     
Epoch 6/8
75110/75110 [==============================] - 7s - loss: 1.7593     
Epoch 7/8
75110/75110 [==============================] - 7s - loss: 1.6914     
Epoch 8/8
75110/75110 [==============================] - 7s - loss: 1.6349     


In [62]:
def get_next_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = np.array(idxs)[np.newaxis,:]
    p = model.predict(arrs)[0]
    return chars[np.argmax(p)]

In [63]:
get_next_keras('this is ')

'a'

## Returning Sequences

### Create Inputs

In [64]:
#input is same, output is just that moved by one character
c_out_dat = [[idx[i+n] for i in xrange(1, len(idx)-cs, cs)] for n in range(cs)]

In [65]:
ys = [np.stack(c[:-2]) for c in c_out_dat]

In [66]:
[xs[n][:cs] for n in range(cs)]

[array([[40],
        [ 1],
        [33],
        [ 2],
        [72],
        [67],
        [73],
        [ 2]]), array([[42],
        [ 1],
        [38],
        [44],
        [ 2],
        [ 9],
        [61],
        [73]]), array([[29],
        [43],
        [31],
        [71],
        [54],
        [ 9],
        [58],
        [61]]), array([[30],
        [45],
        [ 2],
        [74],
        [ 2],
        [76],
        [67],
        [58]]), array([[25],
        [40],
        [73],
        [73],
        [76],
        [61],
        [24],
        [71]]), array([[27],
        [40],
        [61],
        [61],
        [68],
        [54],
        [ 2],
        [58]]), array([[29],
        [39],
        [54],
        [ 2],
        [66],
        [73],
        [33],
        [ 2]]), array([[ 1],
        [43],
        [73],
        [62],
        [54],
        [ 2],
        [72],
        [67]])]

### Create Model

In [67]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax', name='output')

In [68]:
#vector of -s as input:
inp1 = Input(shape=(n_fac,), name='zeros')
hidden = dense_in(inp1)

In [69]:
outs = []

for i in range(cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden], mode='sum')
    #each layer now has an output
    outs.append(dense_out(hidden))

In [71]:
model = Model([inp1] + [c[0] for c in c_ins], outs)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [72]:
zeros = np.tile(np.zeros(n_fac), (len(xs[0]), 1))
zeros.shape

(75110, 42)

In [74]:
model.fit([zeros]+xs, ys, batch_size=64, nb_epoch=12)

Epoch 1/12
75110/75110 [==============================] - 12s - loss: 20.1684 - output_loss_1: 2.7168 - output_loss_2: 2.5726 - output_loss_3: 2.5218 - output_loss_4: 2.4854 - output_loss_5: 2.4838 - output_loss_6: 2.4633 - output_loss_7: 2.4730 - output_loss_8: 2.4516    
Epoch 2/12
75110/75110 [==============================] - 13s - loss: 17.9028 - output_loss_1: 2.5128 - output_loss_2: 2.3532 - output_loss_3: 2.2407 - output_loss_4: 2.1790 - output_loss_5: 2.1661 - output_loss_6: 2.1465 - output_loss_7: 2.1645 - output_loss_8: 2.1399    
Epoch 3/12
75110/75110 [==============================] - 13s - loss: 17.3150 - output_loss_1: 2.4966 - output_loss_2: 2.3312 - output_loss_3: 2.1829 - output_loss_4: 2.0957 - output_loss_5: 2.0686 - output_loss_6: 2.0462 - output_loss_7: 2.0601 - output_loss_8: 2.0337    
Epoch 4/12
75110/75110 [==============================] - 14s - loss: 16.9512 - output_loss_1: 2.4908 - output_loss_2: 2.3223 - output_loss_3: 2.1515 - output_loss_4: 2.0429 - ou

### Test Model

In [75]:
def get_nexts(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict([np.zeros(n_fac)[np.newaxis,:]] + arrs)
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [76]:
get_nexts(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 't', ' ', 'p', 'n', ' ']

In [77]:
get_nexts(' part of')

[' ', 'p', 'a', 'r', 't', ' ', 'o', 'f']


['t', 'o', 'r', 'e', ' ', 'o', 'f', ' ']

### Keras Sequence Model

In [78]:
n_hidden, n_fac, cs, vocab_size

(256, 42, 8, 86)

In [80]:
model=Sequential([
    Embedding(vocab_size, n_fac, input_length=cs),
    SimpleRNN(n_hidden, return_sequences=True, activation='relu', inner_init='identity'),
    TimeDistributed(Dense(vocab_size, activation='softmax'))
])

In [81]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_13 (Embedding)         (None, 8, 42)         3612        embedding_input_2[0][0]          
____________________________________________________________________________________________________
simplernn_2 (SimpleRNN)          (None, 8, 256)        76544       embedding_13[0][0]               
____________________________________________________________________________________________________
timedistributed_1 (TimeDistribut (None, 8, 86)         22102       simplernn_2[0][0]                
Total params: 102,258
Trainable params: 102,258
Non-trainable params: 0
____________________________________________________________________________________________________


In [82]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [83]:
xs[0].shape

(75110, 1)

In [88]:
x_rnn = np.stack(np.squeeze(xs), axis=1) #removes 1D entries from data
y_rnn = np.atleast_3d(np.stack(ys, axis=1))

In [86]:
x_rnn.shape, y_rnn.shape

((75110, 8), (75110, 8, 1))

In [87]:
model.fit(x_rnn, y_rnn, batch_size=64, nb_epoch=8)

Epoch 1/8
75110/75110 [==============================] - 11s - loss: 2.4294    
Epoch 2/8
75110/75110 [==============================] - 11s - loss: 1.9987    
Epoch 3/8
75110/75110 [==============================] - 11s - loss: 1.8825    
Epoch 4/8
75110/75110 [==============================] - 11s - loss: 1.8222    
Epoch 5/8
75110/75110 [==============================] - 11s - loss: 1.7843    
Epoch 6/8
75110/75110 [==============================] - 10s - loss: 1.7583    
Epoch 7/8
75110/75110 [==============================] - 11s - loss: 1.7383    
Epoch 8/8
75110/75110 [==============================] - 11s - loss: 1.7228    


In [89]:
def get_nexts_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arr = np.array(idxs)[np.newaxis,:]
    p = model.predict(arr)[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [90]:
get_nexts_keras(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 'n', ' ', 't', 'n', ' ']

### One-hot Sequence Model with Keras

In [95]:
model=Sequential([
        SimpleRNN(n_hidden, return_sequences=True, input_shape=(cs, vocab_size),
                  activation='relu', inner_init='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
model.compile(loss='categorical_crossentropy', optimizer=Adam())

In [96]:
oh_ys = [to_categorical(o, vocab_size) for o in ys]
oh_y_rnn = np.stack(oh_ys, axis=1)

oh_xs = [to_categorical(o,vocab_size) for o in xs]
oh_x_rnn = np.stack(oh_xs, axis=1)

oh_y_rnn.shape, oh_x_rnn.shape

((75110, 8, 86), (75110, 8, 86))

In [97]:
model.fit(oh_x_rnn, oh_y_rnn, batch_size=64, nb_epoch=8)

Epoch 1/8
75110/75110 [==============================] - 12s - loss: 2.4440    
Epoch 2/8
75110/75110 [==============================] - 11s - loss: 2.0392    
Epoch 3/8
75110/75110 [==============================] - 11s - loss: 1.9242    
Epoch 4/8
75110/75110 [==============================] - 11s - loss: 1.8592    
Epoch 5/8
75110/75110 [==============================] - 11s - loss: 1.8158    
Epoch 6/8
75110/75110 [==============================] - 12s - loss: 1.7848    
Epoch 7/8
75110/75110 [==============================] - 11s - loss: 1.7607    
Epoch 8/8
75110/75110 [==============================] - 11s - loss: 1.7415    


In [98]:
def get_nexts_oh(inp):
    idxs = np.array([char_indices[c] for c in inp])
    arr = to_categorical(idxs, vocab_size)
    p = model.predict(arr[np.newaxis,:])[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [99]:
get_nexts_oh(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 'n', ' ', 'i', 'n', ' ']

### Keras Stateful Model

In [100]:
bs = 64

In [101]:
model=Sequential([
    Embedding(vocab_size, n_fac, input_length=cs, batch_input_shape=(bs,8)),
    BatchNormalization(),
    LSTM(n_hidden, return_sequences=True, stateful=True),
    TimeDistributed(Dense(vocab_size, activation='softmax'))
])

In [103]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [104]:
mx = len(x_rnn)//bs*bs #ensure inputs, outputs are even multiple of batch size b/c fixed batch shap

In [105]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False)

Epoch 1/4
75072/75072 [==============================] - 54s - loss: 2.2282    
Epoch 2/4
75072/75072 [==============================] - 52s - loss: 1.9741    
Epoch 3/4
75072/75072 [==============================] - 52s - loss: 1.8975    
Epoch 4/4
75072/75072 [==============================] - 51s - loss: 1.8518    


In [106]:
model.optimizer.lr=1e-4

In [107]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False)

Epoch 1/4
75072/75072 [==============================] - 53s - loss: 1.8191    
Epoch 2/4
75072/75072 [==============================] - 54s - loss: 1.7932    
Epoch 3/4
75072/75072 [==============================] - 52s - loss: 1.7714    
Epoch 4/4
75072/75072 [==============================] - 53s - loss: 1.7525    


In [108]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False)

Epoch 1/4
75072/75072 [==============================] - 53s - loss: 1.7355    
Epoch 2/4
75072/75072 [==============================] - 58s - loss: 1.7205    
Epoch 3/4
75072/75072 [==============================] - 58s - loss: 1.7066    
Epoch 4/4
75072/75072 [==============================] - 57s - loss: 1.6938    


### Theano RNN

In [109]:
n_input = vocab_size
n_output = vocab_size

In [110]:
def init_wgts(rows, cols):
    scale = math.sqrt(2/rows)
    return shared(normal(scale=scale, size=(rows, cols)).astype(np.float32))
def init_bias(rows):
    return shared(np.zeros(rows, dtype=np.float32))

In [111]:
def wgts_and_bias(n_in, n_out):
    return(init_wgts(n_in, n_out), init_bias(n_out))
def id_and_bias(n):
    return shared(np.eye(n, dtype=np.float32)), init_bias(n)

In [112]:
t_inp = T.matrix('inp')
t_outp = T.matrix('outp')
t_h0 = T.vector('h0')
lr = T.scalar('lr')

all_args = [t_h0, t_inp, t_outp, lr]

In [113]:
W_h = id_and_bias(n_hidden)
W_x = wgts_and_bias(n_input, n_hidden)
W_y = wgts_and_bias(n_hidden, n_output)
w_all = list(chain.from_iterable([W_h, W_x, W_y]))

In [114]:
def step(x, h, W_h, b_h, W_x, b_x, W_y, b_y):
    #calc hidden activ
    h = nnet.relu(T.dot(x,W_x) + b_x + T.dot(h,W_h) + b_h)
    # calculate output activ
    y = nnet.softmax(T.dot(h, W_y) + b_y)
    return h, T.flatten(y, 1)

In [115]:
[v_h, v_y], _ = theano.scan(step, sequences=t_inp, 
                            outputs_info=[t_h0, None], non_sequences=w_all)

/home/dbashir/.local/lib/python2.7/site-packages/ipykernel/__main__.py:5: UserWarning: DEPRECATION: If x is a vector, Softmax will not automatically pad x anymore in next releases. If you need it, please do it manually. The vector case is gonna be supported soon and the output will be a vector.


In [116]:
error = nnet.categorical_crossentropy(v_y, t_outp).sum()
g_all = T.grad(error, w_all)

In [118]:
def upd_dict(wgts, grads, lr):
    return OrderedDict({w: w-g*lr for (w,g) in zip(wgts,grads)})
upd = upd_dict(w_all, g_all, lr)

In [119]:
fn = theano.function(all_args, error, updates=upd, allow_input_downcast=True)

In [ ]:
X = oh_x_rnn
Y = oh_y_rnn
X.shape, Y.shape

((75110, 8, 86), (75110, 8, 86))

In [ ]:
err=0.0; l_rate=0.01
for i in range(len(X)): 
    err+=fn(np.zeros(n_hidden), X[i], Y[i], l_rate)
    if i % 1000 == 999: 
        print ("Error:{:.3f}".format(err/1000))
        err=0.0

Error:25.208
Error:21.546
Error:20.917
Error:19.913
Error:18.840
Error:19.225
Error:19.087
Error:18.465
Error:17.955
Error:18.210
Error:17.500
Error:17.656
Error:18.415
Error:17.320
Error:16.731
Error:17.841
Error:17.397
Error:17.192
Error:16.871
Error:16.721
Error:16.569
Error:16.515
Error:16.711
Error:16.182
Error:16.815
Error:16.641
Error:16.021
Error:16.310
Error:16.277
Error:16.479
Error:16.743


In [ ]:
f_y = theano.function([t_h0, t_inp], v_y, allow_input_downcast=True)

In [ ]:
pred = np.argmax(f_y(np.zeros(n_hidden), X[6]), axis=1)

In [ ]:
act = np.argmax(X[6], axis=1)

In [ ]:
[indices_char[o] for o in act]

In [ ]:
[indices_char[o] for o in pred]